# Regresión lineal

Vamos a hacer un proyecto para predecir la cuota de un partido, es decir si la cuota si gana el equipo local, la cuota si gana el equipo visitante o la cuota si quedan empate;  dados el ID de los equipos locales y visitantes.

Se trata de un problema de regresión lineal ya que queremos un precedir un valor.

Principalmente, lo que hemos hecho es descargar nuestra base de datos SQLite3 en excel, con estos excel los hemos analizado y limpiado. Tras realizar la limpieza, hemos visto que teníamos muy pocos datos, por lo tanto hemos ampliado estos datos con métodos estadísticos, utilizando la base da datos inicial.

## Importamos las librerías

Importamos las librerías a utilizar:

* numpy
* pandas
* matplotlib
* seaborn
* pickle

In [114]:
import pickle

import numpy as np
import pandas as pd
import seaborn as sns

from matplotlib import pyplot as plt


## Descargamos los datos

Vamos a tener dos tipos de datos, los que estaban en un csv en la carpeta \datas.
El otro tipo de datos van a estar en una base de datos SQLlite, la cual la hemos transformado a csv

Descargamos primero los datos: `docs\equipo.csv` y `docs\partido.csv`

In [2]:
team = pd.read_csv('equipo.csv')
matches = pd.read_csv('partidos.csv')

In [3]:
team

,Ajax,ajax.png,Paises Bajos,14
0,Atalanta,atalanta.png,Italia,6
1,Atlético,atletico.png,España,11
2,Barcelona,barcelona.png,España,17
3,Bayern,bayern.png,Alemania,16
4,Benfica,benfica.png,Portugal,15
5,Besiktas,besiktas.png,Turquía,2
6,Chelsea,chelsea.png,Inglaterra,1
7,Club Brugge,club_brugge.png,Bélgica,26
8,Dortmund,dortmund.png,Alemania,27
9,Dynamo Kyiv,dynamo_kyiv.png,Ucrania,28


In [4]:
matches

,14/09/2021 16:45,29,2,33,34,33.1
0,14/09/2021 16:45,31,19,33,34,33
1,14/09/2021 16:45,25,24,33,34,33
2,14/09/2021 19:00,11,6,33,34,33
3,14/09/2021 19:00,4,5,33,34,33
4,14/09/2021 19:00,16,30,33,34,33
...,...,...,...,...,...,...
90,08/12/2021 20:00,5,4,33,34,33
91,08/12/2021 20:00,19,31,33,34,33
92,08/12/2021 20:00,2,29,33,34,33
93,08/12/2021 20:00,24,25,33,34,33


Descargamos los demás datos que estaban en la base de datos: `\bookmaker.db`. Los hemos descargado en .xlsx

In [6]:
apuesta = pd.read_excel('apuestas.xlsx')
clientes = pd.read_excel('clientes.xlsx')
cuotas = pd.read_excel('cuotas.xlsx')
partidos = pd.read_excel('partidos.xlsx')
empresa = pd.read_excel('empresa.xlsx')
equipos = pd.read_excel('equipos.xlsx')


c:\Users\Carlota Martin-Anero\AppData\Local\Programs\Python\Python39\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [7]:
apuesta.head()

,id,fecha,monto,equipo_ganador_id,partido,cliente,ganacia
0,1,2021-12-15,300.0,2.0,1,88,891.0
1,2,2021-12-15,300.0,29.0,1,88,891.0
2,3,2021-12-15,300.0,2.0,1,88,891.0
3,4,2021-12-15,300.0,29.0,1,88,891.0
4,5,2021-12-15,300.0,2.0,1,88,891.0


In [8]:
clientes.head()

,id,nombre,apellido,email,activado
0,1,Lope,Perales,pascualperera@vila.es,1
1,2,Isaura,Canet,gmerino@rocamora.es,1
2,3,Olga,Chaves,amayaprats@llado.com,1
3,4,Celestino,Hoyos,chedelgado@moraleda-pintor.com,1
4,5,Miguel,Milla,inigoprat@gmail.com,1


In [9]:
cuotas.head()

,id,local,empate,visitante,partido_id
0,1,1.40,2.97,5.80,1
1,2,5.73,5.08,3.86,2
2,3,5.45,5.99,5.34,3
3,4,1.95,5.08,4.78,4
4,5,4.44,5.99,4.61,5


In [10]:
partidos.head()

,id,fecha,equipo_local,equipo_visistante,finalizado,ganador
0,1,2021-12-15,29,2,0,NaN
1,2,2021-12-15,31,19,0,NaN
2,3,2021-12-15,25,24,0,NaN
3,4,2021-12-15,11,6,0,NaN
4,5,2021-12-15,4,5,0,NaN


In [11]:
empresa.head()

,id,razon_social,email,activado


In [12]:
equipos.head()

,id,nombre,escudo,pais,puntaje,activado
0,1,Ajax,ajax.png,Paises Bajos,14.0,1
1,2,Atalanta,atalanta.png,Italia,6.0,1
2,3,Atlético,atletico.png,España,11.0,1
3,4,Barcelona,barcelona.png,España,17.0,1
4,5,Bayern,bayern.png,Alemania,16.0,1


## Limpiamos los datos

Primero vamos a limpiar el csv equipos.

In [13]:
equipos

,id,nombre,escudo,pais,puntaje,activado
0,1,Ajax,ajax.png,Paises Bajos,14.0,1
1,2,Atalanta,atalanta.png,Italia,6.0,1
2,3,Atlético,atletico.png,España,11.0,1
3,4,Barcelona,barcelona.png,España,17.0,1
4,5,Bayern,bayern.png,Alemania,16.0,1
5,6,Benfica,benfica.png,Portugal,15.0,1
6,7,Besiktas,besiktas.png,Turquía,2.0,1
7,8,Chelsea,chelsea.png,Inglaterra,1.0,1
8,9,Club Brugge,club_brugge.png,Bélgica,26.0,1
9,10,Dortmund,dortmund.png,Alemania,27.0,1


Hacemos una breve exploración de datos

In [14]:
equipos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id        32 non-null     int64  
 1   nombre    32 non-null     object 
 2   escudo    32 non-null     object 
 3   pais      32 non-null     object 
 4   puntaje   32 non-null     float64
 5   activado  32 non-null     int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 1.6+ KB


In [15]:
equipos.count()

id          32
nombre      32
escudo      32
pais        32
puntaje     32
activado    32
dtype: int64

Miramos la columna 'activado', porque creo que solo hay valores de 1, por lo tando es inútil

In [16]:
equipos['activado'].unique()

array([1], dtype=int64)

Todos los valores de la columna 'activado' son uno, por lo tanto eliminamos esa columna. Tambien eliminamos el escudo, porque no lo necesitamos para nada

In [17]:
equipos = equipos.drop(['activado', 'escudo'], axis=1)
equipos

,id,nombre,pais,puntaje
0,1,Ajax,Paises Bajos,14.0
1,2,Atalanta,Italia,6.0
2,3,Atlético,España,11.0
3,4,Barcelona,España,17.0
4,5,Bayern,Alemania,16.0
5,6,Benfica,Portugal,15.0
6,7,Besiktas,Turquía,2.0
7,8,Chelsea,Inglaterra,1.0
8,9,Club Brugge,Bélgica,26.0
9,10,Dortmund,Alemania,27.0


In [18]:
max_puntaje = equipos[equipos['puntaje'] == equipos['puntaje'].max()]
max_puntaje

,id,nombre,pais,puntaje
28,29,Villarreal,España,30.0


In [19]:
min_puntaje = equipos[equipos['puntaje'] == equipos['puntaje'].min()]
min_puntaje

,id,nombre,pais,puntaje
7,8,Chelsea,Inglaterra,1.0


Exploramos los datos de partidos

In [20]:
partidos

,id,fecha,equipo_local,equipo_visistante,finalizado,ganador
0,1,2021-12-15,29,2,0,NaN
1,2,2021-12-15,31,19,0,NaN
2,3,2021-12-15,25,24,0,NaN
3,4,2021-12-15,11,6,0,NaN
4,5,2021-12-15,4,5,0,NaN
...,...,...,...,...,...,...
92,93,2021-12-15,19,31,0,NaN
93,94,2021-12-15,2,29,0,NaN
94,95,2021-12-15,24,25,0,NaN
95,96,2021-12-15,30,16,0,NaN


In [21]:
partidos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97 entries, 0 to 96
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 97 non-null     int64  
 1   fecha              97 non-null     object 
 2   equipo_local       97 non-null     int64  
 3   equipo_visistante  97 non-null     int64  
 4   finalizado         97 non-null     int64  
 5   ganador            0 non-null      float64
dtypes: float64(1), int64(4), object(1)
memory usage: 4.7+ KB


In [22]:
partidos.count()

id                   97
fecha                97
equipo_local         97
equipo_visistante    97
finalizado           97
ganador               0
dtype: int64

Miramos si hay nulos

In [23]:
equipos.isna().sum()

id         0
nombre     0
pais       0
puntaje    0
dtype: int64

Eliminamos la última columna

In [24]:
partidos = partidos.drop(['ganador'], axis=1)
partidos

,id,fecha,equipo_local,equipo_visistante,finalizado
0,1,2021-12-15,29,2,0
1,2,2021-12-15,31,19,0
2,3,2021-12-15,25,24,0
3,4,2021-12-15,11,6,0
4,5,2021-12-15,4,5,0
...,...,...,...,...,...
92,93,2021-12-15,19,31,0
93,94,2021-12-15,2,29,0
94,95,2021-12-15,24,25,0
95,96,2021-12-15,30,16,0


Miramos si hay nulos

In [25]:
partidos.isna().sum()

id                   0
fecha                0
equipo_local         0
equipo_visistante    0
finalizado           0
dtype: int64

Miramos la columna 'finalizado', porque creo que solo hay valores de 0, por lo tanto es inútil

In [26]:
partidos['finalizado'].unique()

array([0], dtype=int64)

Miramos la columna 'fecha', porque creo que todos tienen la misma fecha, por lo tanto no lo necesitamos

In [27]:
partidos['fecha'].unique()

array(['2021-12-15'], dtype=object)

Todos los valores de la columna 'finalizado' son cero, por lo tanto eliminamos esa columna

In [28]:
partidos = partidos.drop(['finalizado', 'fecha'], axis=1)
partidos

,id,equipo_local,equipo_visistante
0,1,29,2
1,2,31,19
2,3,25,24
3,4,11,6
4,5,4,5
...,...,...,...
92,93,19,31
93,94,2,29
94,95,24,25
95,96,30,16


In [29]:
cuotas

,id,local,empate,visitante,partido_id
0,1,1.40,2.97,5.80,1
1,2,5.73,5.08,3.86,2
2,3,5.45,5.99,5.34,3
3,4,1.95,5.08,4.78,4
4,5,4.44,5.99,4.61,5
...,...,...,...,...,...
93,94,5.80,2.97,1.40,94
94,95,5.34,5.99,5.45,95
95,96,5.91,4.06,3.20,96
96,97,1.40,2.97,5.80,1


In [30]:
cuotas.isna().sum()

id            0
local         0
empate        0
visitante     0
partido_id    0
dtype: int64

Eliminamos las últimas dos filas de Cuotas

In [31]:
cuotas = cuotas.drop([96],axis=0)
cuotas

,id,local,empate,visitante,partido_id
0,1,1.40,2.97,5.80,1
1,2,5.73,5.08,3.86,2
2,3,5.45,5.99,5.34,3
3,4,1.95,5.08,4.78,4
4,5,4.44,5.99,4.61,5
...,...,...,...,...,...
92,93,3.86,5.08,5.73,93
93,94,5.80,2.97,1.40,94
94,95,5.34,5.99,5.45,95
95,96,5.91,4.06,3.20,96


In [32]:
type(cuotas)

pandas.core.frame.DataFrame

In [33]:
type(partidos)

pandas.core.frame.DataFrame

Vamos a revisar el dataFrame de apuesta

In [34]:
apuesta

,id,fecha,monto,equipo_ganador_id,partido,cliente,ganacia
0,1,2021-12-15,300.0,2.0,1,88,891.0
1,2,2021-12-15,300.0,29.0,1,88,891.0
2,3,2021-12-15,300.0,2.0,1,88,891.0
3,4,2021-12-15,300.0,29.0,1,88,891.0
4,5,2021-12-15,300.0,2.0,1,88,891.0
5,6,2021-12-15,300.0,2.0,1,88,891.0
6,7,2021-12-15,300.0,29.0,1,88,891.0
7,8,2021-12-15,300.0,NaN,1,88,891.0
8,9,2021-12-15,300.0,NaN,1,88,891.0
9,10,2021-12-15,300.0,NaN,1,88,891.0


In [35]:
apuesta.isna().sum()

id                   0
fecha                0
monto                0
equipo_ganador_id    7
partido              0
cliente              0
ganacia              0
dtype: int64

Como vemos no obtenemos información interesante porque únicamente tenemos el ejemplo de 20 filas, donde lo único interesante es `equipo_ganador_id`, y en 7/20 filas es nulo

Vamos a unir varios DataFrame para enterner mejor la información

In [36]:
unido = pd.concat([partidos, cuotas], axis= 1)
unido

,id,equipo_local,equipo_visistante,id,local,empate,visitante,partido_id
0,1.0,29.0,2.0,1.0,1.40,2.97,5.80,1.0
1,2.0,31.0,19.0,2.0,5.73,5.08,3.86,2.0
2,3.0,25.0,24.0,3.0,5.45,5.99,5.34,3.0
3,4.0,11.0,6.0,4.0,1.95,5.08,4.78,4.0
4,5.0,4.0,5.0,5.0,4.44,5.99,4.61,5.0
...,...,...,...,...,...,...,...,...
93,94.0,2.0,29.0,94.0,5.80,2.97,1.40,94.0
94,95.0,24.0,25.0,95.0,5.34,5.99,5.45,95.0
95,96.0,30.0,16.0,96.0,5.91,4.06,3.20,96.0
96,97.0,29.0,2.0,NaN,NaN,NaN,NaN,NaN


In [37]:
unido.isna().sum()

id                   1
equipo_local         1
equipo_visistante    1
id                   1
local                1
empate               1
visitante            1
partido_id           1
dtype: int64

Eliminamos las últimas dos filas de `unido`

In [38]:
unido = unido.drop([96, 97],axis=0)
unido

,id,equipo_local,equipo_visistante,id,local,empate,visitante,partido_id
0,1.0,29.0,2.0,1.0,1.40,2.97,5.80,1.0
1,2.0,31.0,19.0,2.0,5.73,5.08,3.86,2.0
2,3.0,25.0,24.0,3.0,5.45,5.99,5.34,3.0
3,4.0,11.0,6.0,4.0,1.95,5.08,4.78,4.0
4,5.0,4.0,5.0,5.0,4.44,5.99,4.61,5.0
...,...,...,...,...,...,...,...,...
91,92.0,5.0,4.0,92.0,4.61,5.99,4.44,92.0
92,93.0,19.0,31.0,93.0,3.86,5.08,5.73,93.0
93,94.0,2.0,29.0,94.0,5.80,2.97,1.40,94.0
94,95.0,24.0,25.0,95.0,5.34,5.99,5.45,95.0


Eliminamos la columnas repetidas de `ID`

In [39]:
unido.columns

Index(['id', 'equipo_local', 'equipo_visistante', 'id', 'local', 'empate',
       'visitante', 'partido_id'],
      dtype='object')

In [40]:
unido.columns = ['id', 'equipo_local', 'equipo_visistante','id_2', 'cuota_local', 'cuota_visitante', 'cuota_empate', 'partido_id']

In [41]:
unido = unido.drop(['id_2', 'partido_id'], axis=1)
unido

,id,equipo_local,equipo_visistante,cuota_local,cuota_visitante,cuota_empate
0,1.0,29.0,2.0,1.40,2.97,5.80
1,2.0,31.0,19.0,5.73,5.08,3.86
2,3.0,25.0,24.0,5.45,5.99,5.34
3,4.0,11.0,6.0,1.95,5.08,4.78
4,5.0,4.0,5.0,4.44,5.99,4.61
...,...,...,...,...,...,...
91,92.0,5.0,4.0,4.61,5.99,4.44
92,93.0,19.0,31.0,3.86,5.08,5.73
93,94.0,2.0,29.0,5.80,2.97,1.40
94,95.0,24.0,25.0,5.34,5.99,5.45


Empezamos el estudio de datos con esta tabla porque es la única que nos proporciona los datos interesantes

In [42]:
unido.describe()

,id,equipo_local,equipo_visistante,cuota_local,cuota_visitante,cuota_empate
count,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000
mean,48.500000,16.500000,16.500000,4.305312,5.194375,4.305313
std,27.856777,9.281561,9.281561,1.366733,0.843506,1.366733
min,1.000000,1.000000,1.000000,1.400000,2.720000,1.400000
25%,24.750000,8.750000,8.750000,3.372500,5.080000,3.372500
50%,48.500000,16.500000,16.500000,4.855000,5.340000,4.855000
75%,72.250000,24.250000,24.250000,5.367500,5.800000,5.367500
max,96.000000,32.000000,32.000000,5.990000,5.990000,5.990000


### Creación de datos sintéticos

Primero importamos faker, que es una librería para crear datos sintéticos

In [54]:
import faker
import random

In [55]:
fake = faker.Factory( )

Vamos a crear un dataFrame que contenga los valores de la cuota_local, cuota_visitante y cuota_empate

In [56]:
cuotas = unido[['cuota_local', 'cuota_visitante', 'cuota_empate']]
cuotas

,cuota_local,cuota_visitante,cuota_empate
0,1.40,2.97,5.80
1,5.73,5.08,3.86
2,5.45,5.99,5.34
3,1.95,5.08,4.78
4,4.44,5.99,4.61
...,...,...,...
91,4.61,5.99,4.44
92,3.86,5.08,5.73
93,5.80,2.97,1.40
94,5.34,5.99,5.45


Vamos a calcular la media, la desviación típica y el mínimo y el máximo de cada columna de cuotas

In [69]:
promedios = cuotas.mean()
desviaciones = cuotas.std()
minimos = cuotas.min()
maximos = cuotas.max()

In [116]:
filas = []
for i in range(10000):
    # Genera un valor para la cuota local. Se asegura que esté entre el mínimo y el máximo
    # Utilizamos la función gaussiana para generar valores aleatorios con una distribución normal en torno al promedio y con la desviación estándar
    valor_local = random.gauss(promedios['cuota_local'], desviaciones['cuota_local'])
    valor_local = max(valor_local, minimos['cuota_local'])
    valor_local = min(valor_local, maximos['cuota_local'])
    # Genera un valor para la cuota visitante
    valor_visitante = random.gauss(promedios['cuota_visitante'], desviaciones['cuota_visitante'])
    valor_visitante = max(valor_local, minimos['cuota_visitante'])
    valor_visitante = min(valor_local, maximos['cuota_visitante'])
    # Genera un valor para la cuota empate
    valor_empate = random.gauss(promedios['cuota_empate'], desviaciones['cuota_empate'])
    valor_empate = max(valor_local, minimos['cuota_empate'])
    valor_empate = min(valor_local, maximos['cuota_empate'])
    fila = {
        'id': i,
        'equipo_local': random.randint(1, 32),
        'equipo_visitante': random.randint(1, 32),
        'cuota_local': valor_local,
        'cuota_visitante': valor_visitante,
        'cuota_empate': valor_empate,
    }
    if fila['equipo_local'] != fila['equipo_visitante']: # Evita que el equipo local y el visitante sean el mismo
        filas.append(fila)

df = pd.DataFrame.from_records(filas)
print(df)

        id  equipo_local  equipo_visitante  cuota_local  cuota_visitante  \
0        0            10                 1     4.754592         4.754592   
1        1            24                16     4.303742         4.303742   
2        2            27                 9     4.378414         4.378414   
3        3            28                 7     5.080638         5.080638   
4        4            29                20     5.990000         5.990000   
...    ...           ...               ...          ...              ...   
9687  9995             9                30     2.684052         2.684052   
9688  9996            15                22     5.675144         5.675144   
9689  9997            32                 4     3.616667         3.616667   
9690  9998            27                18     5.255631         5.255631   
9691  9999            12                22     4.049484         4.049484   

      cuota_empate  
0         4.754592  
1         4.303742  
2         4.378414  
3  

## Data splitting

In [71]:
# Separa las características (X) y las etiquetas (y) del conjunto de datos de entrenamiento
X = df.drop(columns= ['cuota_local', 'cuota_visitante', 'cuota_empate'])
y = df[['cuota_local', 'cuota_visitante', 'cuota_empate']]

Separamos los datos en un train and test set utilizando [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html?highlight=train_test_split#sklearn.model_selection.train_test_split). 


In [72]:
from sklearn.model_selection import train_test_split

In [73]:
# Your code here
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

## Scalers/transformers

Creamos un escalador para transformar las variables intependientes

In [74]:
from sklearn.preprocessing import PowerTransformer


pt = PowerTransformer()
pt.fit(X_train)

PowerTransformer()

In [75]:
X_train_scaled = pt.transform(X_train)
X_test_scaled = pt.transform(X_test)

Ahora transformamos las variables dependientes. 

In [76]:
y_train_scaled = np.log(y_train)
y_test_scaled = np.log(y_test)

Ahora definimos una función para guardar el modelo

In [96]:
import pickle

def save_scaler(amount, filename):
    import os
    if not os.path.exists("scalers"):
        os.mkdir("scalers/")
    filename = "scalers/" + filename
    with open(filename, "wb") as f:
        pickle.dump(amount, f)

In [97]:
save_scaler(pt, filename = "pt.pkl")

## Creamos el modelo de regresión lineal

Ahora que tenemos escalados los datos, hacemos el modelo de regresión

In [79]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()

In [80]:
# Your code here
lr.fit(X_train_scaled, y_train_scaled)

LinearRegression()

In [100]:
# Your code here
def save_model(amount, filename):
    import os
    if not os.path.exists("models"):
        os.mkdir("models/")
    filename = "scalers/" + filename
    with open(filename, "wb") as f:
        pickle.dump(amount, f)

In [101]:
save_model(lr, 'lm.pkl')

In [107]:
# Your code here
y_train_pred = np.exp(lr.predict(X_train_scaled))
y_test_pred = np.exp(lr.predict(X_test_scaled))

array([[4.02822903, 4.02822903, 4.02822903],
       [3.99489832, 3.99489832, 3.99489832],
       [4.03156889, 4.03156889, 4.03156889],
       ...,
       [4.01007092, 4.01007092, 4.01007092],
       [3.98300324, 3.98300324, 3.98300324],
       [4.03077494, 4.03077494, 4.03077494]])

In [103]:
# Your code here
lr.coef_

array([[ 3.83760567e-03, -1.09931068e-03, -2.35753528e-05],
       [ 3.83760567e-03, -1.09931068e-03, -2.35753528e-05],
       [ 3.83760567e-03, -1.09931068e-03, -2.35753528e-05]])

In [104]:
# Your code here
lr.intercept_

array([1.38923178, 1.38923178, 1.38923178])

## Error analysis

In [111]:
def model_evaluation(y_train,y_train_pred,y_test,y_test_pred, k = 0):
    '''
    Function to evaluate a regression model and return a dataframe with the error matrics
    in train and test sets
    Inputs:
    X_train_scaled: dataframe with scaled features for the train set
    X_test_scaled: dataframe with scaled features for the test set
    y_train: dataframe dependent features for the train set
    y_train: dataframe dependent features for the test set
    k: number of parameters in the model
    '''

    from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
    for i in range (3):
        ME_train = round(np.mean( y_train[i] - y_train_pred[i] ),2)
        ME_test  = round(np.mean( y_test[i]  - y_test_pred[i] ),2)

        # Computing the mean absolute error (MAE) for the train and test sets.
        MAE_train = round(mean_absolute_error(y_train[i], y_train_pred[i]),2)
        MAE_test  = round(mean_absolute_error(y_test[i] , y_test_pred[i]),2)

        # Computing the mean squared error (MSE) for the train and test sets.
        MSE_train = round(mean_squared_error(y_train[i], y_train_pred[i]),2)
        MSE_test  = round(mean_squared_error(y_test[i] , y_test_pred[i]),2)

        # Computing the root mean squared error (RMSE) for the train and test sets.
        RMSE_train = round(np.sqrt(MSE_train),2)
        RMSE_test  = round(np.sqrt(MSE_test),2)

        # Computing the R2 score for the train and test sets.
        R2_train = round(r2_score(y_train[i], y_train_pred[i]),2)
        R2_test  = round(r2_score(y_test[i] , y_test_pred[i]),2)

        # Computing the adjusted R2 score for the train and test sets.
        adj_R2_train = round(1 - (1 - R2_train)*(len(y_train) - 1)/(len(y_train) - k - 1),2)
        adj_R2_test  = round(1 - (1 - R2_test)*(len(y_test) - 1)/(len(y_test) - k - 1),2)

        # Creating a dataframe with the error metrics.
        df_metrics = pd.DataFrame({'ME':[ME_train, ME_test],
                                        'MAE':[MAE_train, MAE_test],
                                        'MSE':[MSE_train, MSE_test],
                                        'RMSE':[RMSE_train, RMSE_test],
                                        'R2':[R2_train, R2_test],
                                        'adj_R2':[adj_R2_train, adj_R2_test]},
                                        index = ['Train', 'Test'])

    return df_metrics

In [112]:
# Your code here
model_evaluation(y_train, y_train_pred, y_test, y_test_pred)

KeyError: 0

In [61]:
results_train = pd.DataFrame({'Real': y_train, 'Predicted': y_train_pred, 'Set': 'Train'})
results_test = pd.DataFrame({'Real': y_test, 'Predicted': y_test_pred, 'Set': 'Test'})

results = pd.concat([results_train, results_test], index=[0])

ValueError: If using all scalar values, you must pass an index

In [ ]:
results